In [6]:
import cv2
import pickle
import os.path
import numpy as np
from imutils import paths
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers.core import Flatten, Dense
from helpers import resize_to_fit

In [7]:
LETTER_IMAGES_FOLDER = "extracted_letter_images"
MODEL_FILENAME = "captcha_model.hdf5"
MODEL_LABELS_FILENAME = "model_labels.dat"

In [8]:
# initialize the data and labels
data = []
labels = []

# loop over the input images
for image_file in paths.list_images(LETTER_IMAGES_FOLDER):
    # Load the image and convert it to grayscale
    image = cv2.imread(image_file)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Resize the letter so it fits in a 20x20 pixel box
    image = resize_to_fit(image, 20, 20)

    # Add a third channel dimension to the image to make Keras happy
    image = np.expand_dims(image, axis=2)

    # Grab the name of the letter based on the folder it was in
    label = image_file.split(os.path.sep)[-2]

    # Add the letter image and it's label to our training data
    data.append(image)
    labels.append(label)


In [9]:
# scale the raw pixel intensities to the range [0, 1] (this improves training)
data = np.array(data, dtype="float") / 255.0
labels = np.array(labels)

In [10]:
# Split the training data into separate train and test sets
(X_train, X_test, Y_train, Y_test) = train_test_split(data, labels, test_size=0.25, random_state=0)

In [11]:
# Convert the labels (letters) into one-hot encodings that Keras can work with
lb = LabelBinarizer().fit(Y_train)
Y_train = lb.transform(Y_train)
Y_test = lb.transform(Y_test)

In [12]:
# Save the mapping from labels to one-hot encodings.
# We'll need this later when we use the model to decode what it's predictions mean
with open(MODEL_LABELS_FILENAME, "wb") as f:
    pickle.dump(lb, f)

In [13]:
model = Sequential()

# First convolutional layer with max pooling
model.add(Conv2D(20, (5, 5), padding="same", input_shape=(20, 20, 1), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

# Second convolutional layer with max pooling
model.add(Conv2D(50, (5, 5), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

# Hidden layer with 500 nodes
model.add(Flatten())
model.add(Dense(500, activation="relu"))

# Output layer with 32 nodes (one for each possible letter/number we predict)
model.add(Dense(32, activation="softmax"))

# Ask Keras to build the TensorFlow model behind the scenes
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [14]:
# Train the neural network
model.fit(X_train, Y_train, validation_data=(X_test, Y_test), batch_size=32, epochs=10, verbose=1)


Epoch 1/10
909/909 [==============================] - 17s 18ms/step - loss: 0.2601 - accuracy: 0.9367 - val_loss: 0.0273 - val_accuracy: 0.9917
Epoch 2/10
909/909 [==============================] - 17s 18ms/step - loss: 0.0127 - accuracy: 0.9967 - val_loss: 0.0140 - val_accuracy: 0.9958
Epoch 3/10
909/909 [==============================] - 17s 19ms/step - loss: 0.0071 - accuracy: 0.9981 - val_loss: 0.0166 - val_accuracy: 0.9929
Epoch 4/10
909/909 [==============================] - 18s 19ms/step - loss: 0.0049 - accuracy: 0.9986 - val_loss: 0.0094 - val_accuracy: 0.9976
Epoch 5/10
909/909 [==============================] - 17s 19ms/step - loss: 0.0048 - accuracy: 0.9985 - val_loss: 0.0098 - val_accuracy: 0.9979
Epoch 6/10
909/909 [==============================] - 17s 19ms/step - loss: 0.0021 - accuracy: 0.9994 - val_loss: 0.0111 - val_accuracy: 0.9982
Epoch 7/10
909/909 [==============================] - 18s 20ms/step - loss: 0.0034 - accuracy: 0.9989 - val_loss: 0.0094 - val_accuracy:

In [16]:
# Save the trained model to disk
model.save(MODEL_FILENAME)
